In [4]:
from catboost import CatBoostClassifier
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import scipy
from sklearn.model_selection import RandomizedSearchCV
from kneed import KneeLocator
from eli5.sklearn import PermutationImportance
from utils.general_utils import load_data_old, get_cat_feature_names
from utils.model_extensions_utils import FocalLossObjective
from utils.plot_utils import Evaluation
from utils.preprocessing_utils import Boruta_feature_selection, drop_by_correlation
from utils.fe_utils import get_growth_features

In [5]:
df = load_data_old('test.sql')

In [6]:
df

,expiring_arr,expiring_arr_ytd,arr_amount,event_month,event_yr,event_qtr,event_half,event_type,business_unit,sales_business_unit,...,move_to_onprem,move_to_consolidation_migration,opportunity_full_id,renewal_contract,related_contract_end_date,opp_commit,account_id_renewal,product_renewal_id,insert_datetime,insert_date
0,NaN,NaN,NaN,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Monthly Subscription,NaN,Cloud,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01tw0000004u5DY,2021-08-25 07:02:00.759071,2021-08-25
1,-2950.0,-2950.0,-2950.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,EMEA,EMEA,...,NaN,NaN,006w000000oE98SAAS,8001r000006G4lJ,2018-01-15,False,001w000001Pzv5t,01tw0000004CuUy,2021-08-25 07:02:00.759071,2021-08-25
2,-5500.0,-5500.0,-5500.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,NaN,NaN,...,NaN,NaN,006w000000oEJNxAAO,NaN,2018-01-19,False,001w000001CP8KP,01tw0000004CuUy,2021-08-25 07:02:00.759071,2021-08-25
3,-29500.0,-29500.0,-29500.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,NaN,NaN,...,NaN,NaN,006w000000oEkh1AAC,8001r000006G4p6,2018-01-11,False,001w000001TGRla,01tw0000004CuVD,2021-08-25 07:02:00.759071,2021-08-25
4,-37300.0,-37300.0,-37300.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,NaN,NaN,...,NaN,NaN,006w000000oFvzMAAS,8001r000006G4SR,2018-09-28,False,0012000000uD2Er,01tw0000004CuVD,2021-08-25 07:02:00.759071,2021-08-25
5,-5500.0,-5500.0,-5500.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,NaN,NaN,...,NaN,NaN,006w000000oEBjoAAG,8001r000006G4YF,2018-01-08,False,001w000001Pz3ej,01tw0000004CuUy,2021-08-25 07:02:00.759071,2021-08-25
6,-2950.0,-2950.0,-2950.0,2018-01-01,2018-01-01,2018-01-01,2018-01-01,Customer Success,EMEA,EMEA,...,NaN,NaN,006w000000okmxOAAQ,8001r000006Fwa8,2018-01-20,False,0012000000t89BU,01tw0000004CuUy,2021-08-25 07:02:00.759071,2021-08-25
